In [1]:
!pip install pyquaternion

In [5]:
from bokeh.io import output_notebook, show, curdoc
curdoc().theme = 'light_minimal'
output_notebook()
from bokeh.plotting import figure
 
import numpy as np
import numpy.matlib
from numpy import random

from scipy.integrate import solve_ivp
from scipy.optimize import minimize, fminbound
from scipy.interpolate import pchip_interpolate

import copy

from pyquaternion import Quaternion

import math

# Attitude Determiniation and Control Systems

## pFDA Model

In [6]:
def pfda(t, x, u, p):
  '''pFDA dynamics'''
  return np.array([x[1] + p[0]/p[1] * u, -1/p[1] * (x[1] + p[0]/p[1] * u)]).flatten()

pa = [80e-6, 0.5]   # pFDA parameters
ua = 1              # control

# solve the initial value problem
sol = solve_ivp(pfda, [0, 5], [0, 0], args=(ua, pa), first_step=0.1, max_step=0.1)

# grab simulation output and create output vector from simulated state vector
t = sol.t
xa = sol.y
ya = copy.deepcopy(xa)
ya[1,:] = ya[1,:] + pa[0]/pa[1]*ua